# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [171]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise
    
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype("double")
        r_s = r_s.astype("double")
        a = (l_s - (np.square(l_c) / l_s).sum(axis=1).reshape(-1, 1))
        b = (r_s - (np.square(r_c) / r_s).sum(axis = 1).reshape(-1, 1))
        c = (a + b) / (l_s[0] + r_s[0])
        return c
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype("double")
        r_s = r_s.astype("double")
        return ((-np.nan_to_num(l_c * (np.log(l_c) - np.log(l_s)), copy=False).sum(axis=1))
                + (-np.nan_to_num(r_c * (np.log(r_c) - np.log(r_s)), copy=False).sum(axis = 1))) / (l_s[0] + r_s[0])

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype("double")
        r_s = r_s.astype("double")
        return ((l_s - np.max(l_c, axis=1).reshape(-1, 1))
                + (r_s - np.max(r_c, axis=1).reshape(-1, 1))) / (l_s[0] + r_s[0])

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return xrange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # x - одномерный массив значений одной из фич
        # y - целевой признак
        # Сортирует массивы x и y
        # Считает количество различных классов
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # Отрезает слева и справа по self.min_samples_split
        # Среди оставшейся части ищет все несовпадение классов между соседними элементами
        # в r_border_ids запишет индексы в массиве sorted_y на правые элемент найденых пар  
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # Считает растояние между двумя соседними парами неравных элементов
        # Заполняет таблицу one_hot_code в соответствии с класами левых элементов пары
        # Умножает каждую строку one_hot_code на количество подряд идущих элементов и сохраняет в class_increments
        # к class_increments[0] прибавляются значения классов для первых min_samples_split элементов
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # заменено "y" на "sorted_y"
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)  
        
        # Что делает этот блок кода?
        # l_class_count - i строка содержит сумму i-1 строк таблицы class_increments
        # что равносильно количеству элементов каждого класса слева при разбиению по i границе 
        # r_class_count - i строка содержит разницу np.bincount(y) и i строки l_class_count
        # аналогично l_class_count
        # l_sizes - размеры левых классов
        # r_sizes - размеры правых классов
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
        
        # Что делает этот блок кода?
        # выбирает оптимальное разбиение соответствующее выбранной мере неопределённости
        # gs - массив значений меры для каждого разбиения
        # idx - id оптимального разбиения
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
        # Что делает этот блок кода?
        # возвращает значение меры и критерий разбиения
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
    def __make_leaf(self, class_count, max_class):
            p = class_count/float(len(class_count))
            return (self.LEAF_TYPE, max_class, p)
    def __fit_node(self, x, y, node_id, depth):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        if (len(y) == 0):
            return
        
        class_count = np.bincount(y, minlength=self.num_class)
        max_class = class_count.argmax()
        if (depth == self.max_depth
           or len(y) <= self.min_samples_split*2
           or class_count[max_class] / float(len(y)) >= self.sufficient_share):
            self.tree[node_id] = self.__make_leaf(class_count, max_class)
            return
        
        feature_ids = self.get_feature_ids(x.shape[1])
        min_id = feature_ids[0]
        min_i = np.inf
        min_t = 0
        
        for feature in feature_ids:
            i, t = self.__find_threshold(x[:, feature], y)
            if i <= min_i:
                min_i = i
                min_t = t
                min_id = feature
        xl, xr, yl, yr = self.__div_samples(x, y, min_id, min_t)
        if(len(yl) == 0 or len(yr) == 0):
            self.tree[node_id] = self.__make_leaf(class_count, max_class)
            return
        
        self.tree[node_id] = (self.NON_LEAF_TYPE, min_id, min_t)
        self.__fit_node(xl, yl, node_id * 2 + 1, depth + 1)
        self.__fit_node(xr, yr, node_id * 2 + 2, depth + 1)
        
    
    def fit(self, x, y):
        self.tree = dict()
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [173]:
a = np.square(a)
b = np.array([2, 1, 2, 5]).reshape(-1, 1)
a / b

array([[ 0,  2],
       [49, 25],
       [ 0,  2],
       [ 9,  5]])

In [67]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [89]:
x = df.as_matrix(columns=df.columns[1:])
#x = np.unique(x, axis=0)
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [187]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [188]:
np.unique(y)

array([0, 1], dtype=int64)

## Проверка скорости работы

In [189]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.470999956131
1.10800004005


In [190]:
len(my_clf.tree)

407

## Проверка качества работы

In [194]:
gkf = KFold(n_splits=5, shuffle=True)

In [195]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.977384218841
0.979546021452
0.977633657604
0.978589839528
0.976427056916


In [196]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.891577284443
0.893863806436
0.892616612622
0.889332335578
0.892445848751
